## Random Forest Regression With Pipeline And Hyperparameter Tuning

In [1]:
import seaborn as sns
df = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [2]:
# independent and dependent features
X = df.drop(labels=['total_bill'], axis=1)
y = df.total_bill

In [3]:
y.head()

0    16.99
1    10.34
2    21.01
3    23.68
4    24.59
Name: total_bill, dtype: float64

In [4]:
X.head()

,tip,sex,smoker,day,time,size
0,1.01,Female,No,Sun,Dinner,2
1,1.66,Male,No,Sun,Dinner,3
2,3.50,Male,No,Sun,Dinner,3
3,3.31,Male,No,Sun,Dinner,2
4,3.61,Female,No,Sun,Dinner,4


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3, test_size=0.20)

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer   # Handle missing values
from sklearn.preprocessing import StandardScaler # Feature scaling
from sklearn.preprocessing import OneHotEncoder # Categorical to numerical
from sklearn.compose import ColumnTransformer

In [17]:
categorical_col = ['sex', 'smoker', 'day', 'time']
numerical_col = ['tip', 'size']

In [18]:
# feature engineering automation
# numerical pipeline
num_pipeline = Pipeline(
    steps = [
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

cat_pipeline = Pipeline(
    steps = [
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehotencoder', OneHotEncoder())
    ]
)

In [19]:
preprocessor = ColumnTransformer([
    ('num_pipeline', num_pipeline, numerical_col),
    ('cat_pipeline', cat_pipeline, categorical_col)
])

In [20]:
preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['tip', 'size']),
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder())]),
                                 ['sex', 'smoker', 'day', 'time'])])

In [21]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [22]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor

In [38]:
## Model Training Automation
models = {
    'Random Forest' : RandomForestRegressor(),
    # 'Logistic Regression' : LogisticRegression(),
    'Decision Tree' : DecisionTreeRegressor(),
}

In [39]:
from sklearn.metrics import r2_score

In [40]:
def eval_Models(X_train, y_train, X_test, y_test, models):
    report = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        model.fit(X_train, y_train)
        y_test_pred = model.predict(X_test)
        test_model_score = r2_score(y_test, y_test_pred)
        report[list(models.keys())[i]] = test_model_score
    return report

In [41]:
eval_Models(X_train, y_train, X_test, y_test, models)

{'Random Forest': 0.6055058841316456, 'Decision Tree': 0.40827539172015204}

## Hyperparameter Tuning

In [42]:
regressor = RandomForestRegressor()

In [59]:
params={'max_depth':[3,5,10,None],
              'n_estimators':[100,200,300],
               'criterion':['squared_error', 'absolute_error', 'friedman_mse', 'poisson']
              }

In [44]:
from sklearn.model_selection import RandomizedSearchCV

In [60]:
cv=RandomizedSearchCV(regressor,param_distributions=params,cv=5,verbose=3)
cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=poisson, max_depth=5, n_estimators=200;, score=0.512 total time=   0.3s
[CV 2/5] END criterion=poisson, max_depth=5, n_estimators=200;, score=0.547 total time=   0.2s
[CV 3/5] END criterion=poisson, max_depth=5, n_estimators=200;, score=0.494 total time=   0.2s
[CV 4/5] END criterion=poisson, max_depth=5, n_estimators=200;, score=0.316 total time=   0.2s
[CV 5/5] END criterion=poisson, max_depth=5, n_estimators=200;, score=0.386 total time=   0.2s
[CV 1/5] END criterion=squared_error, max_depth=10, n_estimators=300;, score=0.514 total time=   0.3s
[CV 2/5] END criterion=squared_error, max_depth=10, n_estimators=300;, score=0.430 total time=   0.3s
[CV 3/5] END criterion=squared_error, max_depth=10, n_estimators=300;, score=0.500 total time=   0.3s
[CV 4/5] END criterion=squared_error, max_depth=10, n_estimators=300;, score=0.292 total time=   0.3s
[CV 5/5] END criterion=squared_error, max_depth=10, n_es

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(),
                   param_distributions={'criterion': ['squared_error',
                                                      'absolute_error',
                                                      'friedman_mse',
                                                      'poisson'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   verbose=3)

In [61]:
cv.best_params_

{'n_estimators': 300, 'max_depth': 5, 'criterion': 'friedman_mse'}

In [62]:
cv.best_score_

0.4611393357595035